In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
from tqdm import tqdm
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, BatchNormalization, Activation, Input
from keras.models import Sequential, Model
from sklearn.metrics import confusion_matrix, classification_report
from keras import regularizers
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

warnings.filterwarnings("ignore")

2023-06-11 19:59:30.165827: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 19:59:30.460681: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 19:59:30.464696: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 19:59:33.304552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
res = 216

def load_data(directory):
    images = []
    labels = []
    for folder in tqdm(os.listdir(directory)):
        label = folder
        data_path = os.path.join(directory, folder)
        if not os.path.isdir(data_path):
            continue
        for filename in tqdm(os.listdir(data_path), position=0, leave=True, desc=f"Loading images from {directory.split(os.sep)[-2]} folder"):
            img = cv2.imread(os.path.join(data_path, filename))
            if img is None:
                continue
            img = cv2.resize(img, (res, res))
            img = np.array(img)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


In [3]:
x_train, y_train = load_data('chest_xray/chest_xray/train/')
x_test, y_test = load_data('chest_xray/chest_xray/test/')

100%|██████████| 3/3 [00:06<00:00,  2.22s/it]


In [4]:
y_train = np.where(y_train == 'NORMAL', 0, 1)
y_test = np.where(y_test == 'NORMAL', 0, 1)

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# assuming you've got these 4 arrays from somewhere
# x_train, x_test, y_train, y_test
print('AHAHAHAHAHAHAAH')
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, stratify = y_train)

# normalize the data
x_train = x_train[0 : 700] / 255.0
x_test = x_test[0 : 700] / 255.0

# convert labels to one-hot vectors
y_train = to_categorical(y_train[0 : 700])
y_test = to_categorical(y_test[0 : 700])
print('AHAHAHAHAHAHAAH')
# define the model structure
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Sequential()
model.add(base_model)
model.add(Dense(y_train.shape[1], activation='softmax'))  # y_train.shape[1] is num_classes
print('BABABABABABABAABABAB')
# do not train the base model, only train the classifier layer
base_model.trainable = False

# compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train[0 : 700], y_train[0 : 700], validation_data=(x_test[0 : 700], y_test[0 : 700]), epochs=5)
print('CCCCCCCCCCCCCCCCCCCCCCCCCCCCC')
# after initial training, you may want to fine-tune the entire model with a lower learning rate
'''base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train[0 : 700], y_train[0 : 700], validation_data=(x_test[0 : 700], y_test[0 : 700]), epochs=5)
print('CCCCCCCCCCCCCCCCCCCCCCCCCCCCC')'''


AHAHAHAHAHAHAAH
AHAHAHAHAHAHAAH
BABABABABABABAABABAB


2023-06-11 20:00:55.283152: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 391910400 exceeds 10% of free system memory.


Epoch 1/5


2023-06-11 20:01:00.529115: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2023-06-11 20:01:00.625507: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2023-06-11 20:01:00.692678: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 99123200 exceeds 10% of free system memory.
2023-06-11 20:01:00.767698: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.


22/22 [==============================] - ETA: 0s - loss: 0.6210 - accuracy: 0.7200

: 

: 